In [ ]:
#import relevant libraries
import os
import sys

import numpy as np
import pandas as pd
import sys; sys.path
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multitest import multipletests as fdr
from matplotlib import colors
from scipy import stats

from sklearn.metrics import explained_variance_score, r2_score, classification_report
from sklearn.linear_model import Ridge, RidgeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, GroupKFold, GroupShuffleSplit, StratifiedKFold
from sklearn.svm import SVC
from scipy import stats
from sklearn.utils import shuffle
from datetime import datetime


import warnings
warnings.filterwarnings('ignore')

In [ ]:
# read and clean up ABCD data
# set base dirctories
ABCD_base_dir   = 'PATH_TO_DIR'
ABCD_results_dir   = 'PATH_TO_DIR'

#load subj fc data
ABCD_fc= pd.read_csv(os.path.join(ABCD_base_dir, 'ABCD_rsfc_withsubcort_5260subj.csv'), header=None)
ABCD_fc = ABCD_fc.T
ABCD_subj = pd.read_csv(os.path.join(ABCD_base_dir, 'subjects_5260.txt'), header=None, names='s')
ABCD_fc.insert(0, "subjectkey", ABCD_subj, True)
ABCD_fc = ABCD_fc.sort_values(by='subjectkey', ascending=True)

# load subj demo and clinical data
ABCD_gender_p = pd.read_csv(os.path.join(ABCD_base_dir, 'abcd_pgi01.csv'), header=0)
ABCD_gender_y = pd.read_csv(os.path.join(ABCD_base_dir, 'abcd_ygi01.csv'), header=0)

#drop duplicate header rows
header_row = 0
ABCD_gender_p = ABCD_gender_p.drop(header_row)
ABCD_gender_y = ABCD_gender_y.drop(header_row)

In [ ]:
#select subjects for whom we have FC data
ABCD_gender_p = ABCD_gender_p[ABCD_gender_p.subjectkey.isin(ABCD_subj.s)]
ABCD_gender_y = ABCD_gender_y[ABCD_gender_y.subjectkey.isin(ABCD_subj.s)]

In [ ]:
#sort data using subjectkey so it's all in the same order
ABCD_gender_p = ABCD_gender_p.sort_values(by='subjectkey', ascending=True)
ABCD_gender_y = ABCD_gender_y.sort_values(by='subjectkey', ascending=True)

In [ ]:
#select variables of interest
ABCD_gender_y = ABCD_gender_y[['subjectkey', 'sex', 'eventname',
                               'gish_m1_y', 'gish_m2_y', 
                               'gish_f1_y', 'gish_f2_y', 
                               'gish_m3_y', 'gish_m4_y', 
                               'gish_f3_y', 'gish_f4_y']]


ABCD_gender_p = ABCD_gender_p[['subjectkey', 'sex', 'eventname',
                               'gish_m1_p', 'gish_m2_p', 'gish_m3_p', 'gish_m4_p', 
                               'gish_m5_p', 'gish_m6_p', 'gish_m7_p', 'gish_m8_p', 
                               'gish_m9_p', 'gish_m10_p', 'gish_m11_p', 'gish_m12_p', 
                               'gish_m13_p', 'gish_m14_p', 
                               'gish_f1_p', 'gish_f2_p', 'gish_f3_p', 'gish_f4_p',
                               'gish_f5_p', 'gish_f6_p', 'gish_f7_p', 'gish_f8_p', 
                               'gish_f9_p', 'gish_f10_p', 'gish_f11_p', 'gish_f12_p', 
                               'gish_f13_p', 'gish_f14_p']]


In [ ]:
#create separate variables for self-report and parent-report data
ABCD_gender_p_f1 = ABCD_gender_p[(ABCD_gender_p.eventname == '1_year_follow_up_y_arm_1')]
ABCD_gender_y_f1 = ABCD_gender_y[(ABCD_gender_y.eventname == '1_year_follow_up_y_arm_1')]

In [ ]:
#separate data by assigned sex
ABCD_gender_p_f1_f = ABCD_gender_p_f1[(ABCD_gender_p_f1.sex =='F')]
ABCD_gender_y_f1_f = ABCD_gender_y_f1[(ABCD_gender_y_f1.sex =='F')]
ABCD_gender_p_f1_m = ABCD_gender_p_f1[(ABCD_gender_p_f1.sex =='M')]
ABCD_gender_y_f1_m = ABCD_gender_y_f1[(ABCD_gender_y_f1.sex =='M')]


In [ ]:
#organize all data
subj_m = ABCD_gender_y_f1_m.subjectkey
subj_f = ABCD_gender_y_f1_f.subjectkey

gender_y_f1_m = ABCD_gender_y_f1_m[['gish_m1_y', 'gish_m2_y', 'gish_m3_y', 'gish_m4_y']]
gender_y_f1_f = ABCD_gender_y_f1_f[['gish_f1_y', 'gish_f2_y', 'gish_f3_y', 'gish_f4_y']]


gender_p_f1_m = ABCD_gender_p_f1_m[['gish_m1_p', 'gish_m2_p', 'gish_m3_p', 'gish_m4_p',
                                    'gish_m5_p', 'gish_m6_p', 'gish_m7_p', 'gish_m8_p', 
                                    'gish_m10_p', 'gish_m12_p', 'gish_m13_p', 'gish_m14_p']]

gender_p_f1_f = ABCD_gender_p_f1_f[['gish_f1_p', 'gish_f2_p', 'gish_f3_p', 'gish_f4_p',
                                    'gish_f5_p', 'gish_f6_p', 'gish_f7_p', 'gish_f8_p',
                                    'gish_f10_p', 'gish_f12_p', 'gish_f13_p', 'gish_f14_p']]



In [ ]:
#remove subjects with missing data
nanmask = np.isnan(np.double(gender_y_f1_m)).any(axis=1)
gender_y_m = gender_y_f1_m[~nanmask]
gender_p_m = gender_p_f1_m[~nanmask]
subj_m = subj_m[~nanmask]

nanmask = np.isnan(np.double(gender_y_f1_f)).any(axis=1)
gender_y_f = gender_y_f1_f[~nanmask]
gender_p_f = gender_p_f1_f[~nanmask]
subj_f = subj_f[~nanmask]


In [ ]:
#remove subjects who did not answer the questions
nanmask = np.isnan(np.double(gender_p_m)).any(axis=1)
gender_y_m = np.double(gender_y_m[~nanmask])
gender_p_m = np.double(gender_p_m[~nanmask])
subj_m = subj_m[~nanmask]


nanmask = np.isnan(np.double(gender_p_f)).any(axis=1)
gender_y_f = np.double(gender_y_f[~nanmask])
gender_p_f = np.double(gender_p_f[~nanmask])
subj_f = subj_f[~nanmask]


mask777 = (gender_p_m>=777)
mask777 = mask777.any(axis=1)
gender_y_m_clean = gender_y_m[~mask777]
gender_p_m_clean = gender_p_m[~mask777]
subj_m = subj_m[~mask777]


mask777 = (gender_p_f>=777)
mask777 = mask777.any(axis=1)
gender_y_f_clean = gender_y_f[~mask777]
gender_p_f_clean = gender_p_f[~mask777]
subj_f = subj_f[~mask777]




In [ ]:
#take the sum to get summary self- and parent- report scores
gender_y_m_sum = np.sum(gender_y_m_clean, axis=1)
gender_y_f_sum = np.sum(gender_y_f_clean, axis=1)
gender_p_m_sum = np.sum(gender_p_m_clean, axis=1)
gender_p_f_sum = np.sum(gender_p_f_clean, axis=1)

In [ ]:
#load data on site 
ABCD_site = pd.read_csv(os.path.join(ABCD_base_dir, 'abcd_lt01.csv'), header=0)
ABCD_site = ABCD_site.drop(header_row)
ABCD_site_f1 = ABCD_site[ABCD_site.eventname == 'baseline_year_1_arm_1']
ABCD_site_f1 = ABCD_site_f1.sort_values(by='subjectkey', ascending=True)
ABCD_site_f1 = ABCD_site_f1[['subjectkey', 'site_id_l']]
ABCD_site_f1.reset_index(inplace=True) 


site_m = ABCD_site_f1[ABCD_site_f1.subjectkey.isin(subj_m)].site_id_l
site_f = ABCD_site_f1[ABCD_site_f1.subjectkey.isin(subj_f)].site_id_l





In [ ]:
#site names
sites = ['Colorado Boulder',
         'Florida International',
         'Laureate Institute',
         'Medical University of South Carolina',
         'Oregon Health and Science University',
         'University of Rochester',
         'Stanford Research Institute International',
         'University of California - Los Angeles',
         'University of California - San Diego',
         'University of Florida',
         'University of Maryland Baltimore',
         'University of Michigan',
         'University of Minnesota',
         'University of Pittsburgh',
         'University of Utah',
         'University of Wisconsin-Milwaukee',
         'Washington University St.Louis',
         'Yale']

In [ ]:
#remove data from specific sites with fewer than 10 subjects
site_mask = site_m.values!=('site22')
site_m = site_m[site_mask]
subj_m = subj_m[site_mask]
gender_y_m_sum = gender_y_m_sum[site_mask]
gender_p_m_sum = gender_p_m_sum[site_mask]

site_mask = site_m.values!=('site19')
site_m = site_m[site_mask]
subj_m = subj_m[site_mask]
gender_y_m_sum = gender_y_m_sum[site_mask]
gender_p_m_sum = gender_p_m_sum[site_mask]

site_mask = site_f.values!=('site22')
site_f = site_f[site_mask]
subj_f = subj_f[site_mask]
gender_y_f_sum = gender_y_f_sum[site_mask]
gender_p_f_sum = gender_p_f_sum[site_mask]

site_mask = site_f.values!=('site19')
site_f = site_f[site_mask]
subj_f = subj_f[site_mask]
gender_y_f_sum = gender_y_f_sum[site_mask]
gender_p_f_sum = gender_p_f_sum[site_mask]



In [ ]:
#load in FC data
ABCD_fc= pd.read_csv(os.path.join(ABCD_base_dir, 'ABCD_rsfc_withsubcort_5260subj.csv'), header=None)
ABCD_fc = ABCD_fc.T
ABCD_subj = pd.read_csv(os.path.join(ABCD_base_dir, 'subjects_5260.txt'), header=None, names='s')
ABCD_fc.insert(0, "subjectkey", ABCD_subj, True)
ABCD_fc = ABCD_fc.sort_values(by='subjectkey', ascending=True)

In [ ]:
#separate by assigned sex
fc_m = ABCD_fc[ABCD_fc.subjectkey.isin(subj_m)]
fc_f = ABCD_fc[ABCD_fc.subjectkey.isin(subj_f)]

fc_m.reset_index(inplace=True) 
fc_m = fc_m.drop(columns=['index', 'subjectkey'])

fc_f.reset_index(inplace=True) 
fc_f = fc_f.drop(columns=['index', 'subjectkey'])

In [ ]:
#set up predictive models
#number of repetitions you want to perform (100 for 'true', 1000 for 'null')
rep = 100

#number of folds you want in the cross-validation
k = 3
#proportion of data you want in your training set and test set
train_size = .80
test_size = 1-train_size

#regression model type
regr = Ridge(normalize=True, max_iter=1000000, solver='lsqr')
#set hyperparameter grid space you want to search through for the model
#adapted from the Thomas Yeo Lab Github: 
#ThomasYeoLab/CBIG/blob/master/stable_projects/predict_phenotypes/He2019_KRDNN/KR_HCP/CBIG_KRDNN_KRR_HCP.m
#alphas = [0, 0.00001, 0.0001, 0.001, 0.004, 0.007, 0.01, 0.04, 0.07, 0.1, 0.4, 0.7, 1, 1.5, 2, 2.5, 3,
#          3.5, 4, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 100, 150, 200, 300, 500, 700, 1000, 2000, 
#          3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

alphas = [0.001, 0.01, 0.1, 1, 2, 3, 4, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 100, 
          150, 200, 300, 500, 700, 1000]

#alphas = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]

#param grid set to the hyperparamters you want to search through
paramGrid ={'alpha': alphas}

#set x data to be the input variable you want to use
X = fc_f.values
Y = gender_p_f_sum.T
site = site_f.values

#number of features 
n_feat = X.shape[1]

pred_name = 'parent_report'
pred_sex = 'f'


In [ ]:
#create variables to store relevant data/outputs
#r^2 - coefficient of determination
r2 = np.zeros([rep])
#explained variance
var = np.zeros([rep])
#correlation between true and predicted (aka prediction accuracy)
corr = np.zeros([rep])
#optimised alpha (hyperparameter)
opt_alpha = np.zeros([rep])
#predictions made by the model
#don't need to save any of these right now
#preds = np.zeros([rep,n_cog,int(np.ceil(X.shape[0]*test_size))])
#true test values for cognition
#cogtest = np.zeros([rep,n_cog,int(np.ceil(X.shape[0]*test_size))])
#feature importance extracted from the model
featimp = np.zeros([rep,n_feat])
#for when the feat weights get haufe-inverted
#featimp_haufe = np.zeros([rep,n_feat])


In [ ]:
#iterate through the diff train/test splits

#alphas = np.load(results_dir + '/fc_alpha_' + pred_name + '_opt.npy') ##only need this for the null ones

for p in range(rep):
    
    #print model # you're on
    print('Model %d' %(p+1))
    
    #Y_shuffle = shuffle(Y, random_state=p) ##only need this for the null ones
    
    #print time
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)
    
    #split into train/test data
    train_inds, test_inds = next(GroupShuffleSplit(test_size=1-train_size, n_splits=1, random_state = p).split(X, groups=site))
    
    #set x values based on indices from split
    x_train = X[train_inds]
    x_test = X[test_inds]
        
    #set y values based on indices from split  
    beh_train = Y[train_inds]
    beh_test = Y[test_inds]
    
    #set y values based on indices from split ##only need this for the null ones
    #beh_train = Y_shuffle[train_inds]
    #beh_test = Y_shuffle[test_inds]
    
    site_train = site[train_inds]
    site_test = site[test_inds] 
    
    #convert y values to to double
    y_train = np.double(beh_train)
    y_test = np.double(beh_test)



    #create variables to store nested CV scores, and best parameters from hyperparameter optimisation
    best_scores = []
    best_params = []
    

        
    #set parameters for inner and outer loops for CV
    cv_split = GroupKFold(n_splits=k)
        
    print ("Optimising Models")
            
    #define regressor with grid-search CV for inner loop
    gridSearch = GridSearchCV(estimator=regr, param_grid=paramGrid, n_jobs=-1, verbose=0, cv=cv_split, scoring='explained_variance')

    #fit regressor to the model, use site ID as group category again
    gridSearch.fit(x_train, y_train, groups=site_train)

    #save parameters corresponding to the best score
    best_params.append(list(gridSearch.best_params_.values()))
    best_scores.append(gridSearch.best_score_)
        
        
    
        
    print ("Evaluating Models")
        
    #save optimised alpha values
    opt_alpha[p] = best_params[best_scores.index(np.max(best_scores))][0]
    
    
    #rand_alpha = np.random.choice(alphas)
    
    #fit optimized models
    model = Ridge(alpha = opt_alpha[p], normalize=True, max_iter=1000000, solver='lsqr')
    #model = Ridge(alpha = rand_alpha, normalize=True, max_iter=1000000, solver='lsqr')

    model.fit(x_train, y_train);
        
        
    #evaluate model
    r2[p]=model.score(x_test,y_test)
    print(r2[p])
        
    #generate predictions in test set
    preds = []
    preds = model.predict(x_test).ravel()
    
        
    #compute explained variance 
    var[p] = explained_variance_score(y_test, preds)
    print(var[p])


    #compute correlation between true and predicted (prediction accuracy)
    corr[p] = np.corrcoef(y_test.ravel(), preds)[1,0]
    print(corr[p])
    
    #print ("Haufe-Transforming Feature Weights")
    cov_x = []
    cov_y = []
    
    #extract feature importance
    featimp[p,:] = model.coef_
    #compute Haufe-inverted feature weights
    cov_x = np.cov(np.transpose(x_train))
    cov_y = np.cov(y_train)
    featimp_haufe[p,:] = np.matmul(cov_x,featimp[p,:])*(1/cov_y)
        

        
    #save results
    np.save((results_dir + '/fc_r2_' + pred_name + pred_sex + '.npy'),r2)
    np.save((results_dir + '/fc_var_' + pred_name + pred_sex + '.npy'),var)
    np.save((results_dir + '/fc_corr_' + pred_name + pred_sex + '.npy'),corr)
    np.save((results_dir + '/fc_alpha_' + pred_name + pred_sex + '.npy'),opt_alpha)
    np.save((results_dir + '/fc_featimp_' + pred_name + pred_sex + '.npy'),featimp)
    np.save((results_dir + '/fc_featimp_haufe_' + pred_name + pred_sex + '.npy'),featimp_haufe_f)

    